# Project Overview

Our project's objective is to achieve precise localization for a car driving within a simulated environment. The task involves ensuring that the car can travel a minimum distance of 170 meters from its starting position while minimizing the distance pose error. Practically, this error should not exceed 1.2 meters.

To accomplish this, we will utilize point clouds extracted from a simulated car equipped with a LIDAR sensor, which provides regular LIDAR scans. Additionally, we have an existing point cloud map, which has been extracted from the CARLA simulator. Localization will be achieved by performing point registration matching between the real-time LIDAR scans and the pre-existing point cloud map. For this we implemented the ICP and NTD localization algorithm respectively.

# Set Up
TODO: Write explaination about the steps to run the code

# ICP Localization

The Iterative Closest Point (ICP) algorithm is used to align two sets of points, typically referred to as the source points \( P \) and the target points \( Q \). In the context of a self-driving car, the target points \( Q \) are provided by a map, while the source points \( P \) come from the car's sensors. The goal is to transform the car's sensor data so it aligns with the map coordinates, specifically aligning the x and y coordinates through translation and rotation. This is achieved by minimizing the sum of squared errors between the points. This is achieved iteratively. 

One iteration includes the following steps:

1. Calculate the centroids (mean points) of both \( P \) and \( Q \).

2. Create a new set \( X \) by subtracting each point in \( P \) by the centroid of \( P \), and create a new set \( Y \) by subtracting each point in \( Q \) by the centroid of \( Q \).

3. Compute the cross covariance matrix \( S \) by multiplying \( X \) with the transpose of \( Y \).

4. Perform SVD on the matrix \( S \) to obtain matrices \( U \), \( \Sigma \), and \( V^T \).

5. 
    - The rotation matrix \( R \) is calculated as \( R = VU^T \).
    - The translation vector \( t \) is calculated as \( t = \mu_Q - R\mu_P \), where \( \mu_Q \) and \( \mu_P \) are the centroids of \( Q \) and \( P \), respectively.

6. We then iterativeley repeat the above steps until the sum of squared errors is minimized to a satisfactory level.

By iteratively updating the transformation, the ICP algorithm aligns the source points \( P \) to the target points \( Q \), effectively mapping the car's sensor data to the map coordinates.

## Module configuration

TODO: Explain the module configuration and selected parameters

## Dataset performance metrics

On the provided dataset, our model demonstrated the following performance metrics:

- **Mean time per frame**: 0.12 seconds
- **Maximal time per frame**: 0.59 seconds
- **Mean lateral error**: 0.07 meters
- **Maximal lateral error**: 0.07 meters


## Code

In [2]:
# TODO: Update paths, remove final part about visualization?

import open3d as o3d
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import pandas as pd
import os

#
# Paths
map_path = "localization/dataset/map.pcd"
frames_path = "localization/dataset/frames"
ground_truth_path = "localization/dataset/ground_truth.csv"

# point cloud
map_cloud = o3d.io.read_point_cloud(map_path)

# ground truth
ground_truth = pd.read_csv(ground_truth_path)

extracted_cars = []

#frame
for i in range(len(ground_truth)):
    frame_file = os.path.join(frames_path, f"frame_{i}.pcd")

    if not os.path.exists(frame_file):
        print(f"Frame file {frame_file} not found.")
        continue

    frame_cloud = o3d.io.read_point_cloud(frame_file)
    frame_cloud_down = frame_cloud.voxel_down_sample(voxel_size=0.05)

    # ground truth
    x, y, z = ground_truth.iloc[i][1:4]
    roll, pitch, yaw = np.deg2rad(ground_truth.iloc[i][4:7])

    # transformation matrix
    r = R.from_euler('xyz', [roll, pitch, yaw])
    rotation_matrix = r.as_matrix()
    translation_vector = np.array([x, y, z])
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = translation_vector

    # transformation
    frame_cloud_down.transform(transformation_matrix)
    extracted_cars.append(frame_cloud_down)

# ICP
threshold = 0.1
initial_transformation = np.eye(4)
maximum_iteration = 100

time_rec = []
lateral_err = []


for i, car_cloud in enumerate(extracted_cars):
    start_time = time.time()

    reg_p2l = o3d.pipelines.registration.registration_icp(
        car_cloud, map_cloud, threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=maximum_iteration)
    )

    end_time = time.time()
    time_rec.append(end_time - start_time)

    # registration error
    lateral_error = o3d.pipelines.registration.evaluate_registration(
        car_cloud, map_cloud, threshold, reg_p2l.transformation
    ).inlier_rmse
    lateral_err.append(lateral_error)

    if lateral_error > 1.2:
        print(f"Lateral error ({lateral_error:.2f} m) is greater than the maximum allowed (1.2 m).")
        break

    # initial transformation
    initial_transformation = reg_p2l.transformation

    print(f"Processed {i + 1}/{len(extracted_cars)} frames.")

# results
print(f"----->>>>>>>>>>  Mean time per frame: {np.mean(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Maximal time per frame: {np.max(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Mean lateral error: {np.mean(lateral_err):.2f} m")
print(f"----->>>>>>>>>>  Maximal lateral error: {np.max(lateral_err):.2f} m")


# aligned car point clouds
merged_cloud = map_cloud
for car_cloud in extracted_cars:
    merged_cloud += car_cloud

o3d.io.write_point_cloud("aligned_car.pcd", merged_cloud)
# # ######################################################### End of solution #########################################################

# After saving the aligned point cloud, you can visualize it using the following code:
# pcd = o3d.io.read_point_cloud("/home/ari/Workplace/JKU/SEM_2/Autonomous_sys/Project3/Localization_Project/localization/Project/ICP/aligned_car.pcd")
# o3d.visualization.draw_geometries([pcd])

# NTD Localization

## Algorithm explaination

Unlike the Iterative Closest Point (ICP) algorithm, which minimizes point-to-point distances, Normal Distribution Transform (NDT) aligns a set of source points to a target point cloud by modeling the target as a set of normal distributions via Multivariate Normal Distribution. Since it is possible that we obtain probability density functions that dont represent our point cloud, we divide the space of the poinclouds into a grid of cells (also called voxels) and calculate the Multivariate Normal Distribution for each cell. 

 #### TODO: Explain the steps in NTD
 


 


1. Initialization: 
Before the alignment , the algorithm initializes parameters such as grid size and point cloud resolution. This is where the NDT class and other helper classes like Pose and Cell are defined to support the main NDT operations.

2. Preprocessing:
The input cloud is set into the NDT grid using set_input_cloud. Here, the point cloud data is divided into a 3D grid of cells. The grid size is determined by the resolution parameter. The input cloud is then transformed into the grid coordinates.

3. Gaussian Distribution Calculation:
Each cell calculates and stores a mean and covariance matrix representing the Gaussian distribution of points within that cell.

4. The target cloud (to be aligned) is iteratively transformed and aligned to the reference model built from the input cloud.For each iteration, the algorithm computes a transformation matrix from the current pose estimation.The transformed point cloud is then used to calculate a score indicating how well it fits the reference model.A Newton-Raphson update step is applied to refine the pose estimation, aiming to maximize the alignment score.

5. Iteration and Convergence:The alignment iterates, updating the pose based on calculated gradients and the Hessian matrix until convergence criteria are met (based on a threshold or a maximum number of iterations).

## Module configuration
TODO: Explain the module configuration and selected parameters 


x_step, y_step, z_step: The resolution of the grid in each dimension, set to 15 units in your case. This resolution defines the size of each cell in the 3D grid.

eps: The convergence threshold, set to 1e-3, determining the minimum change between iterations for the process to be considered as converged.

In the implementation, after setting up the NDT environment with the defined grid size and resolution, we proceed to align a series of extracted car point clouds against a map. The initial pose is assumed to be neutral (Pose(0, 0, 0, 0, 0, 0)), and this iteratively refine this pose based on the NDT alignment results. The lateral error and timing are tracked to evaluate the performance.

In the begining since we didn't have the right configuration for NDT from open3d. we tried to implement the NDT algorithm from scratch. We used the following steps to implement the NDT algorithm from scratch:

1. Initialize the NDT grid with the given resolution and grid size.
2. Preprocess the input point cloud by transforming it into the grid coordinates.
3. Calculate the Gaussian distribution for each cell in the grid.
4. Align the target cloud to the reference model iteratively.
5. Update the pose estimation based on the calculated gradients and Hessian matrix.
6. Repeat the alignment process until convergence criteria are met.

but somehow we were not able to get the right results. the threshold for the lateral error was not met. it was increasesing incrementally. 

Lateral error is : 0.5159613500514074
Processed 949/1014 frames.
Lateral error is : 0.48686348588548367
Processed 950/1014 frames.
Lateral error is : 0.4907630302487757
Processed 951/1014 frames.
Lateral error is : 0.542017974444391
Processed 952/1014 frames.
Lateral error is : 0.5516454626605112
Processed 953/1014 frames.
Lateral error is : 0.5567147416256384
Processed 954/1014 frames.
Lateral error is : 0.5738285602910101
Processed 955/1014 frames.
Lateral error is : 0.611308163787111
Processed 956/1014 frames.
Lateral error is : 0.5971138202110213
Processed 957/1014 frames.
Lateral error is : 0.6755034481165051
Processed 958/1014 frames.
Lateral error is : 0.6468705750935433
Processed 959/1014 frames.
Lateral error is : 0.7296657503271184
Processed 960/1014 frames.
Lateral error is : 0.6938888359361489
Processed 961/1014 frames.
Lateral error is : 0.8619496501455042
Processed 962/1014 frames.
Lateral error is : 0.8766180618122889
Processed 963/1014 frames.
Lateral error is : 0.8429278811469212
Processed 964/1014 frames.
Lateral error is : 1.014644693568059
Processed 965/1014 frames.
Lateral error is : 1.0259737317016615
Processed 966/1014 frames.
Lateral error is : 0.9943898462532704
Processed 967/1014 frames.
Lateral error is : 1.0945663941821813
Processed 968/1014 frames.
Lateral error is : 1.1947511112389448
Processed 969/1014 frames.
Lateral error is : 1.1551949147881855
Processed 970/1014 frames.
Lateral error is : 1.1829704847569473
Processed 971/1014 frames.
Lateral error is : 1.3387442052416239
Lateral error (1.34 m) is greater than the maximum allowed (1.2 m).

we got that we reached the maximum allowed lateral error of 1.2 meters at the frame of 971. 


## Dataset performance metrics

- **Mean time per frame**: 0.08 s
- **Maximal time per frame**: 0.35 s
- **Mean lateral error**: 94.98 m
- **Maximal lateral error**: 182.41 m

## Code

In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd
import os
import time

#### Gaussian Distributions ###
def compute_gaussian_distributions(points, voxel_indices):
    voxel_dict = {}

    for point, index in zip(points, voxel_indices):
        index_tuple = tuple(index)
        if index_tuple not in voxel_dict:
            voxel_dict[index_tuple] = []
        voxel_dict[index_tuple].append(point)

    voxel_mean = {}
    voxel_covariance = {}

    for index in voxel_dict:
        voxel_points = np.array(voxel_dict[index])

        if len(voxel_points) < 2:
            continue

        q = np.mean(voxel_points, axis=0)
        C = np.cov(voxel_points, rowvar=False)
        C += np.eye(C.shape[0]) * 1e-6

        voxel_mean[index] = q
        voxel_covariance[index] = C

    return voxel_mean, voxel_covariance

### Rotation Matrix from RPY ###
def rpy_to_rotation_matrix(roll, pitch, yaw):
    # Calculate cosines and sines
    cy = np.cos(yaw)
    sy = np.sin(yaw)
    cp = np.cos(pitch)
    sp = np.sin(pitch)
    cr = np.cos(roll)
    sr = np.sin(roll)

    # Create rotation matrices
    R_z = np.array([
        [cy, -sy, 0],
        [sy, cy, 0],
        [0, 0, 1]
    ])

    R_y = np.array([
        [cp, 0, sp],
        [0, 1, 0],
        [-sp, 0, cp]
    ])

    R_x = np.array([
        [1, 0, 0],
        [0, cr, -sr],
        [0, sr, cr]
    ])

    # Combine rotations
    R = np.dot(R_z, np.dot(R_y, R_x))
    return R

### Voxelization ###
def voxelize(points, voxel_size):
    min_bounds = np.min(points, axis=0)
    voxel_indices = np.floor((points - min_bounds) / voxel_size).astype(int)
    return voxel_indices, min_bounds

def compare_transformations(computed, ground_truth):
    translation_gt = ground_truth[[' x', ' y', ' z']].values
    rpy_gt = ground_truth[[' roll', ' pitch', ' yaw']].values
    rotation_gt = rpy_to_rotation_matrix(*rpy_gt)

    gt_transform = np.eye(4)
    gt_transform[:3, :3] = rotation_gt
    gt_transform[:3, 3] = translation_gt

    # print("Ground Truth Transformation Matrix:\n", gt_transform)
    # print("Difference in Transformation:\n", computed - gt_transform)

### NDT Registration ###
def ndt_registration(source_points, voxel_mean, voxel_covariance, voxel_size, min_bounds, max_iterations=30, tolerance=1e-6):
    transformation = np.eye(4)

    def get_voxel_index(point):
        return tuple(np.floor((point - min_bounds) / voxel_size).astype(int))

    for iteration in range(max_iterations):
        transformed_points = (transformation[:3, :3] @ source_points.T + transformation[:3, 3].reshape(-1, 1)).T

        likelihood = 0
        gradient = np.zeros((6,))

        for point in transformed_points:
            voxel_index = get_voxel_index(point)
            if voxel_index in voxel_mean:
                q = voxel_mean[voxel_index]
                C = voxel_covariance[voxel_index]

                diff = point - q
                likelihood += np.exp(-0.5 * diff.T @ np.linalg.inv(C) @ diff)
                # Compute the gradient based on the likelihood (not fully implemented here)

        # Update the transformation using the gradient (simplified, gradient step needs implementation)
        # transformation_update = compute_transformation_update(gradient)
        # transformation = transformation @ transformation_update

        if np.linalg.norm(gradient) < tolerance:
            break

    return transformation

### Load Point Cloud ###
def load_point_cloud(filename):
    return o3d.io.read_point_cloud(filename)

def apply_voxel_downsampling(pcd, voxel_size=0.05):
    return pcd.voxel_down_sample(voxel_size=voxel_size)

def load_ground_truth(filename):
    return pd.read_csv(filename)

#### Main ####
# Project directory setup

# TODO: Change directory set up below

# project_dir = os.path.dirname(os.path.abspath(__file__))
# project_dir = '/'.join(project_dir.split('/')[:-2])
project_dir = os.getcwd()
dataset_dir = os.path.join(project_dir, 'localization/dataset')
frames_dir = sorted(os.listdir(os.path.join(dataset_dir, 'frames')))




# Load map and ground truth
map_pcd = load_point_cloud(os.path.join(dataset_dir, 'map.pcd'))
map_points = np.asarray(map_pcd.points)

# Load ground truth
ground_truth = load_ground_truth(os.path.join(dataset_dir, 'ground_truth.csv'))

voxel_size = 1.0
voxel_indices, min_bounds = voxelize(map_points, voxel_size)
voxel_mean, voxel_covariance = compute_gaussian_distributions(map_points, voxel_indices)

# Placeholder for initial transformation (identity)
initial_transformation = np.eye(4)

merged_pcd = o3d.geometry.PointCloud()
time_rec = []
lateral_err = []
# Loop through all frames
for i, frame_file in enumerate(frames_dir):
    frame_pcd = load_point_cloud(os.path.join(dataset_dir, 'frames', frame_file))
    frame_points = np.asarray(frame_pcd.points)

    start_time = time.time()
    transformation = ndt_registration(frame_points, voxel_mean, voxel_covariance, voxel_size, min_bounds)
    end_time = time.time()

    aligned_filename = os.path.join(dataset_dir, 'aligned_frames', f'aligned_{frame_file}.pcd')
    frame_pcd.transform(transformation)
    merged_pcd += frame_pcd


    # Calculate lateral error (example calculation)
    gt_transform = ground_truth.iloc[i]
    translation_gt = gt_transform[[' x', ' y', ' z']].values
    rpy_gt = gt_transform[[' roll', ' pitch', ' yaw']].values
    rotation_gt = rpy_to_rotation_matrix(*rpy_gt)

    gt_transform_matrix = np.eye(4)
    gt_transform_matrix[:3, :3] = rotation_gt
    gt_transform_matrix[:3, 3] = translation_gt

    # Compute the lateral error (difference in transformation)
    difference = np.linalg.norm(transformation[:3, 3] - gt_transform_matrix[:3, 3])

    computation_time = float("{:.2f}".format(end_time - start_time))
    time_rec.append(computation_time)

    lat_err = float("{:.2f}".format(difference))
    lateral_err.append(lat_err)

    print(f"Frame {i}/{len(frames_dir)}: Lateral error = {difference:.2f} meters, Computation time = {end_time - start_time:.2f} seconds")

    # Update initial transformation for next iteration (could be based on odometry or other)
    initial_transformation = transformation

    # Optional: Compare transformations
    compare_transformations(transformation, gt_transform)

# Print metrics
print(f"----->>>>>>>>>>  Mean time per frame: {np.mean(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Maximal time per frame: {np.max(time_rec):.2f} s")
print(f"----->>>>>>>>>>  Mean lateral error: {np.mean(lateral_err):.2f} m")
print(f"----->>>>>>>>>>  Maximal lateral error: {np.max(lateral_err):.2f} m")

# Optional: Voxel downsample the merged point cloud to reduce noise and size
merged_pcd = merged_pcd.voxel_down_sample(voxel_size=0.05)

# Save the merged point cloud
merged_filename = os.path.join(dataset_dir, 'merged_map.pcd')
o3d.io.write_point_cloud(merged_filename, merged_pcd)


#pcd = o3d.io.read_point_cloud("/home/ari/Workplace/JKU/SEM_2/Autonomous_sys/Project3/Autonomous_vehicles_ue_3/localization/Project/ICP/aligned_car.pcd")
#o3d.visualization.draw_geometries([pcd])


# Analysis


### Comparison of ICP and NDT Algorithms

#### Error Analysis

**ICP Mean lateral error:** 0.07 m
**NTD Mean lateral error:** 94.98 m

**ICP Maximal lateral error:** 0.07 m
**NTD Maximal lateral error:** 182.41 m

**TODO**: Update if NDT Code is updated
We observed lower error values in the ICP implementation compared to the NTD implementation. 



#### Computation Time

**ICP Mean time per frame:** 0.12 s, 
**NTD Mean time per frame:** 0.08 s, 

**ICP Maximal time per frame:** 0.59 s
**NTD Maximal time per frame:** 0.35 s

In our project NDT shows advantages in terms of computational efficiency, requiring less time per frame on average and exhibiting lower maximum computation times. Algorithmic complexity also favors NDT in terms of scalability and computational load, especially in larger datasets or real-time applications. The observed superior computational efficiency is also confirmed by the literature: 
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8906233/#acm213521-bib-0017
https://publish.mersin.edu.tr/index.php/igd/article/view/387
 **TODO: Add proper citation** 


### Complexity

TODO: Compare the complexity of the algorithms
 


